## Mathematical Operations without built in `+` / `-` / `*` / `/`

- An interesting application of bit manipulation techniques is how it can be applied to perform mathematical operations without the use of built in symbols

- I'm not sure when you'll ever want to do something like this, but it's a fun exercise nonetheless

### Example (Addition)

- Let's imagine we want to do `15 + 11`

- In binary:
    - `11` = `1011`
    - `15` = `1111`

- We know that the answer is
    - `15 + 11 = 26` = `11010`

- Let's think through what it means to add in binary
    - Cases
        1. For any position `i`, if value for both binary numbers are 1, then the final answer must have a value of `1` in position `i+1`, and 0 in position `i`
            - That is `001` + `001` = `010` by definition (because each i+1 position is double the value of each `i` position)

        2. For any position `i`, if value for both binary numbers are 0, then the final answer must have a value of 0 in position `i`

        3. For any position `i`, if there is one 1 and one 0, then the final answer must have a value of 1 in position `i`

    - This simply means, for any position `i`, we need an `XOR` operator for the binary representation

- Let's try the `XOR` approach
    - `1011 ^ 1111 = 0100` = 4 

- Hey, `XOR` gives us the wrong answer!! Why?
    - Because, we are neglecting an important part of the addition operation; we may have values to carry forward!
    - For example, if we XOR two 1s, the addition will give us a 2, which will require us to carry a value forward one position

- How do we operationally do this carry forward using binary?
    - Simple! We only ever need to carry stuff forward when we have two 1s. In all other cases, there is no carry forward
    - That is, we take a binary `&` to find the indices where carry forward is needed
    - Then, to add them with our `XOR` result, we simply do a rightshift to move the carry values to the correct indices
        - i.e. if I have two 1s in position `i`, I need to add 1 in position `i+1`

- We keep doing this until the binary `&` gives us 0, and that's when we know there is nothign else to carry forward. Then we just return the `XOR`

In [44]:
a = 9
b = 11

while b:
    a, b = a^b, (a&b)<<1
a

20

### Example (Subtraction)

- Let's imagine we want to do `22 - 6`

- In binary:
    - `22` = `10110`
    - `9` = `01001`

- We know that the answer is
    - `22 - 9 = 13` = `01101`

- Let's think through what it means to subtract in binary
    - Cases
        1. For any position `i`, if value for both binary numbers are equal (1,1 or 0,0), then the final answer must have a value of `0` in position `i`
            - `001` - `001` = `000` 
            - `000` - `000` = `000` 

        2. For any position `i`, if value is (1, 0), then the final answer must have a value of 1 in position `i`
            - `001` - `000` = `001` 

        3. For any position `i`, if there is (0, 1), then the final answer must have a value of 1 in position `i`, but subtracting 1 from position `i+1`
            - `101` - `010` = `011`

- So again, the idea of taking `a ^ b` remains!

- Unlike addition, instead of "carrying forward", we need to "borrow"
    - Recall that in addition, each time we see a (1,1), we take a logical `&`, then left shift it by 1

- This time, each time we see a (0,1), we left shift by 1

- Working through `22-9`
    - Iter 1: 
        - a = `10110 ^ 01001 = 11111`
        - b = `(~10110 & 01001) << 1 = (01001 & 01001) << 1 = (01001) << 1 = 10010`
    - Iter 2:
        - a = `11111 ^ 10010 = 01101`
        - b = `(~11111 & 10010) << 1 = (00000 & 10010) << 1 = (00000) << 1 = 000000`
    - End, a = 13

- Intuitively, each "borrow" corrects 1 position of (0,1)
- In the worst case, you only need to propagate the one leftwards $N$ times, where $N$ is the length of the binary string

In [53]:
a = 22
b = 9 

while b:
    ## Perform xor, same as addition
    ## But unlike addition, where we applied & and left shifted to find the carry forward 
    ## we find all cases of (0,1) and left shift to find the subtraction
    print(a,b)
    a, b = a ^ b, (~a & b) << 1 
    print(a,b)

a

22 9
31 18
31 18
13 0


13

### Example (Multiplication)

- This is a bit different from addition/subtraction

- But idea is similar, we go bit by bit, and perform some action depending on what we see!

- Let's imagine we want to do 29 * 13
    - 29 --> 11101
    - 13 --> 01101

- We traverse 13 from right to left, going bit by bit:
    - Let thirteen = '01101'
    - thirteen[4] = 1
        - This implies there is a 2^0 value
        - Find 29 * 2^0 = 29
    - thirteen[3] = 0
        - Skip
    - thirteen[2] = 1
        - There is a 2^2 = 4 value
        - 29 * 2^2 = 116
    - thirteen[1] = 1
        - There is a 2^3 = 8 value
        - 29 * 2^3 = 232
    - thirteen[0] = 0
        - Skip

- 232 + 116 + 29 = 377 = 13 * 29


In [62]:
a = 13
b = 29

res = 0
power = 0
while b > 0:
    
    ## Get the least signinificant bit of b
    lsb = b & 1
    if lsb:
        res += a << power
    
    ## Have accounted for least significant bit, get rid of it
    b >>= 1
    power += 1

res
    

377

### Example (Division)

- Let's imagine we want to do 29 / 6
    - 35 --> 100011
    - 6 --> 000110

- Instead of trying to multiply 35 by each non-zero bit of 6, we now want to figure out the highest multiple of 6 that can fit 35

- Iteration 1
    - Left shift 6 until it exceeds 35
        - 000110 << 1 --> 001100 = 12. NOT LARGER THAN 35, continue. shift = 1
        - 001100 << 1 --> 011000 = 24. NOT LARGER THAN 35, continue. shift = 2
        - 011000 << 1 --> 110000 = 48. LARGER THAN 35, stop.
    
    - Shift = 2
        - That is, the most number of times we double 6 before it exceeds 35 is 2

    - Set remainder = 35-24 = 11 
    - Set quotient += 2^shift = 2^2 = 4

- Iteration 2
    - Left shift 6 until it exceeds 11
        - 000110 << 1 --> 001100 = 12. LARGER THAN 11, Stop
    
    - Shift = 0
        - That is, the most number of times we double 6 before it exceeds 11 is 0

    - Set remainder = 11-6 = 5
    - Set quotient += 2^shift = 2^0 = 4 + 1 = 5

- Iteration 3
    - Remainder < divisor, terminate 


In [70]:
a = 40
b = 6

remainder = a
quotient = 0

while remainder >= b:
    shift = 0
    divisor = b
    while divisor < remainder:
        divisor <<= 1
        shift += 1
    
    quotient += 2**(shift-1)
    remainder -= 2**(shift-1) * b

(quotient, remainder)

0 40
4 16
4 16
6 4


(6, 4)

In [1]:
def divide(a, b):
    remainder = a
    quotient = 0

    while remainder >= b:
        shift = 0
        divisor = b
        
        # Shift the divisor to the left until it's larger than the remainder
        while divisor <= remainder:
            divisor <<= 1
            shift += 1

        # Calculate the quotient using bitwise OR and subtract the multiple of divisor
        '''
        This takes some explaining.
        In this while loop, we always divide by the largest possible value at the start. 
            - i.e. in 35/6, we divide by 24 at the start)
        Every subsequent increment of quotient will come from less significant bits
        So here, we want to find the most significant increment of the quotient that is a power of 2 (as per our division strategy)
            - i.e. in 35/6, we divide by 6 * 2^2 at the start, so the quotient will have a bit flipped at the 3rd bit, or at 1 << 2 bit
        By taking an `|`, every bit that we have flipped earlier remains flipped, and every new (smaller) bit will get flipped
        '''
        quotient |= (1 << (shift - 1))
        
        # remainder -= (1 << (shift - 1)) * b
        remainder -= b << (shift - 1)

    return quotient, remainder

divide(35,6)

(5, 5)